In [1]:
import numpy as nm
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import xgboost as xgb
import datetime as dt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [3]:
labels= pd.read_csv("train_labels.csv")
labels

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1


In [4]:

final_data=pd.DataFrame()
data= pd.read_csv('train_data.csv',nrows=100000)
data["S_2"]=pd.to_datetime(data["S_2"])
data["Year-Month"] = data["S_2"].dt.to_period('M')
data1=data.groupby('customer_ID').apply(lambda x: x.sample(n=1)).reset_index(drop = True)
final_data=pd.concat([final_data,pd.merge(labels,data1, how= 'inner', on='customer_ID')])


final_data

final_data.to_csv("Final_data-Step3.csv")




In [ ]:
# temp["S_2"]=pd.to_datetime(temp["S_2"])
# temp["Year-Month"] = temp["S_2"].dt.to_period('M')
# temp1=temp.groupby(['customer_ID','Year-Month'])
# ttemp2 = temp1.first()
# temp = pd.read_csv('train_data.csv',nrows=100)
# temp.dtypes
# temp

In [ ]:
# temp

In [5]:
dtypes=final_data.dtypes

dtypelist = list()

for col in dtypes.index:
    if dtypes[col] == 'object':
        dtypelist.append(col)
        
print(dtypelist)

# final_data['D_63'].value_counts()
# final_data['D_64'].value_counts()


['customer_ID', 'D_63', 'D_64']


In [6]:

size = final_data.size
size

1592448

In [7]:
one_hot = pd.get_dummies(final_data[['D_63', 'D_64']])

# Combine the one hot encoded data with the original dataframe
final_data = pd.concat([final_data, one_hot], axis=1)

# Drop the original categorical columns
final_data.drop(['D_63', 'D_64', 'customer_ID'], axis=1, inplace=True)



In [19]:
# dtypes=final_data.dtypes

# dtypelist1 = list()

# for col in dtypes.index:
#     if dtypes[col] == 'object':
#         dtypelist1.append(col)
        
# print(dtypelist1)

# final_data['D_64'].value_counts()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [8]:
final_data.dtypes

final_data.replace('', nm.nan, inplace=True)


In [9]:
final_data.to_csv("XGBoost_Input_final.csv")

In [10]:
value = final_data['D_42'][1]

print(value)

nan


In [11]:
final_data['Year-Month']

0       2017-03
1       2017-05
2       2017-12
3       2017-07
4       2017-05
         ...   
8289    2018-01
8290    2017-09
8291    2017-10
8292    2017-11
8293    2017-04
Name: Year-Month, Length: 8294, dtype: period[M]

In [12]:
#test-train-test split

train_start_date = '2017-05'
train_end_date = '2018-01'

# Define the start and end dates for the test 1 set
test_start_date = '2017-03'
test_end_date = '2017-04'

# Define the start and end dates for the test 2 setYear-Month
test1_start_date = '2018-02'
test1_end_date = '2018-03'

# Split the data into training and test sets
train_final_data = final_data[(final_data['Year-Month'] >= train_start_date) & (final_data['Year-Month'] <= train_end_date)]
test_final_data = final_data[(final_data['Year-Month'] >= test_start_date) & (final_data['Year-Month'] <= test_end_date)]
test2_final_data = final_data[(final_data['Year-Month'] >= test_start_date) & (final_data['Year-Month'] <= test_end_date) ]         

In [13]:
print(len(train_final_data))

5624


In [14]:
train_final_data

train_final_data['target'].dtypes


dtype('int64')

In [15]:
print(len(test_final_data))

1106


In [16]:
print(len(test2_final_data))

1106


In [17]:
display(train_final_data)

,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,D_64_-1,D_64_O,D_64_R,D_64_U
1,0,2017-05-07,0.922563,0.567403,0.033713,1.007497,0.005594,0.094655,0.003162,0.010466,...,0,1,0,0,0,0,0,1,0,0
2,0,2017-12-12,0.872014,0.007485,0.001891,0.816083,0.006484,NaN,0.007051,0.007042,...,0,1,0,0,0,0,0,0,1,0
3,0,2017-07-16,0.598278,0.000660,0.045268,1.007780,0.008866,0.162771,0.004137,0.007301,...,0,1,0,0,0,0,0,1,0,0
4,0,2017-05-30,0.937438,0.003936,0.003352,0.814304,0.002594,NaN,0.000545,0.003142,...,0,1,0,0,0,0,0,1,0,0
6,0,2017-05-12,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8288,1,2017-12-21,0.305306,0.035043,0.064785,0.094911,0.509138,0.225223,0.007922,0.174662,...,0,1,0,0,0,0,0,0,0,1
8289,1,2018-01-25,0.526164,0.035330,0.714960,0.026882,0.251946,0.209647,0.005900,1.007715,...,0,1,0,0,0,0,0,1,0,0
8290,0,2017-09-03,0.920667,0.009949,0.003489,1.000081,0.003225,0.144085,0.008304,0.007060,...,1,0,0,0,0,0,0,0,0,1
8291,0,2017-10-26,0.854344,0.001644,0.008311,1.006577,0.005204,0.152868,0.003385,0.007144,...,0,1,0,0,0,0,0,0,1,0


In [18]:

# Extract the input features and target variable from the training data
X_train = train_final_data.drop(columns=['target','S_2', 'Year-Month'])
y_train = train_final_data['target']

# Extract the input features and target variable from the test data
X_test = test_final_data.drop(columns=['target','S_2', 'Year-Month'])
y_test = test_final_data['target']

# Extract the input features and target variable from the test data 1
X_test1 = test2_final_data.drop(columns=['target','S_2', 'Year-Month'])
y_test1 = test2_final_data['target']

# Define the XGBoost model
model = xgb.XGBClassifier(random_state=52)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the accuracy of the model on the test data
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test data:", accuracy)

# Make predictions on the test data 1
y_pred1 = model.predict(X_test1)

# Evaluate the accuracy of the model on the test data 1
accuracy1 = accuracy_score(y_test1, y_pred1)
print("Accuracy on test data 1:", accuracy1)



Accuracy on test data: 0.8426763110307414
Accuracy on test data 1: 0.8426763110307414


In [19]:
#Feature_Importance
importance = model.feature_importances_

print(importance)

feature_importance_df = pd.DataFrame(list(zip(X_train.columns, importance)), columns=['feature', 'importance'])

# sort the dataframe in descending order of importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# print the dataframe
print(feature_importance_df)

feature_importance_df.to_csv("Feature_Importance_1.csv")


[0.1626387  0.00424849 0.00921448 0.00883168 0.00808287 0.01122728
 0.00660475 0.00851075 0.01444073 0.00932448 0.00783467 0.0093553
 0.0091998  0.00521929 0.00534365 0.00583465 0.00661815 0.00708495
 0.0050751  0.00212861 0.00675377 0.00490209 0.00717978 0.00326799
 0.01428428 0.00760644 0.00677576 0.00318214 0.00175152 0.00306725
 0.00757013 0.00893366 0.00516434 0.0029179  0.00200098 0.00516158
 0.00575678 0.00278079 0.00442245 0.00869351 0.00288137 0.00485144
 0.00442572 0.00471564 0.0038928  0.00331718 0.0040894  0.00984422
 0.00231422 0.00593374 0.00852369 0.00484426 0.00409181 0.00556565
 0.00995853 0.00257473 0.0034214  0.00335262 0.00078181 0.00403701
 0.00213452 0.00406884 0.00488692 0.00276116 0.00794101 0.00607152
 0.00337992 0.0039349  0.00632035 0.00858639 0.         0.00355757
 0.00170707 0.02281533 0.00610925 0.00412518 0.00296658 0.00713835
 0.00173779 0.00249741 0.00353952 0.00440062 0.00698007 0.0036595
 0.00332888 0.0011132  0.00387082 0.00268629 0.00488321 0.003669

In [20]:
# set the hyperparameters
params = {
    'n_estimators': 300,
    'learning_rate': 0.5,
    'max_depth': 4,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'scale_pos_weight': 5
}

# create the XGBClassifier object with the hyperparameters
model1 = xgb.XGBClassifier(**params)

# train the model on the training set
model1.fit(X_train, y_train)

# # make predictions on the test set
# y_pred = model.predict(X_test)

# print(y_pred)

importance = model1.feature_importances_

print(importance)



[0.02936756 0.00342817 0.01087152 0.01389714 0.00532627 0.00974959
 0.00659307 0.02499316 0.00729906 0.00631196 0.00653549 0.00705377
 0.00460882 0.00666691 0.00560447 0.00350025 0.01191912 0.02852331
 0.01090805 0.003406   0.00610509 0.0104621  0.0144779  0.00522925
 0.00403794 0.01270267 0.00616485 0.00742617 0.00366866 0.00720762
 0.00476529 0.00405162 0.00576198 0.00481099 0.00428056 0.00624297
 0.0032061  0.00902877 0.00322555 0.00681742 0.00373821 0.00200095
 0.00323556 0.0100659  0.00548002 0.00355718 0.00377675 0.04559926
 0.00404742 0.00487064 0.00606404 0.00262177 0.00296716 0.00366112
 0.00489926 0.00380222 0.         0.00607889 0.00044936 0.00475937
 0.00442768 0.00505834 0.004437   0.00620319 0.00780143 0.00345156
 0.00449188 0.00531604 0.00308327 0.00332847 0.01798632 0.00205782
 0.00553811 0.01112732 0.01122383 0.00458459 0.00237779 0.01543164
 0.00396288 0.00679526 0.00427073 0.00340503 0.00253584 0.00441491
 0.00316062 0.0024856  0.00392787 0.00383605 0.00498916 0.0044

In [21]:
feature_importance = pd.DataFrame(list(zip(X_train.columns, importance)), columns=['feature', 'importance'])

# sort the dataframe in descending order of importance
feature_importance = feature_importance_df.sort_values(by='importance', ascending=False)

# print the dataframe
print(feature_importance)

     feature  importance
0        P_2    0.162639
73      D_75    0.022815
8       D_42    0.014441
24       B_9    0.014284
5        S_3    0.011227
..       ...         ...
191  D_63_XZ    0.000000
186  D_63_CL    0.000000
194   D_64_R    0.000000
192  D_64_-1    0.000000
195   D_64_U    0.000000

[196 rows x 2 columns]


In [22]:
feature_importance.to_csv("Feature_Importance2.csv")

In [23]:
df_selected_features = feature_importance[feature_importance['importance'] >= 0.005]
print(df_selected_features)

   feature  importance
0      P_2    0.162639
73    D_75    0.022815
8     D_42    0.014441
24     B_9    0.014284
5      S_3    0.011227
..     ...         ...
13     B_5    0.005219
93    B_28    0.005180
32     S_6    0.005164
35     S_7    0.005162
18    D_49    0.005075

[65 rows x 2 columns]


In [24]:
# Subset X_train to include only selected features
X_train_selected = X_train.loc[:, df_selected_features['feature'].tolist()]

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 300],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.5, 0.8],
    'colsample_bytree': [0.5, 1.0],
    'scale_pos_weight': [1, 5, 10]
}

# Create an XGBoost classifier object
xgb_model = xgb.XGBClassifier()

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_selected, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                   

In [28]:
grid_search.cv_results_
# print(grid_search.best_params_)
# print(grid_search.best_estimator_)
# print(grid_search.best_score_)
# print(grid_search.scorer_)

{'mean_fit_time': array([ 4.87001438,  6.58376961,  5.40040126,  7.44230342,  5.68734579,
         6.2300909 ,  9.31461039, 11.43909159,  9.9663856 , 12.70912414,
        10.12870884, 12.65480161, 27.26184964, 36.82918224, 28.90752358,
        36.25621796, 26.92090993, 35.29790478,  4.49719515,  5.79143744,
         4.80619941,  6.08834872,  4.5770577 ,  5.92872896,  8.6633698 ,
        11.04839993,  8.78125095, 11.57927208,  8.82455025, 11.55603123,
        26.02121882, 35.65726843, 27.99626722, 35.83239603, 31.20142026,
        41.81059513, 11.23974805, 11.90434184,  9.00915732, 12.03497486,
         9.06889853, 11.92843227, 17.33994555, 22.65446143, 18.19008255,
        23.08683472, 18.00203843, 23.66974611, 50.58684387, 67.77756066,
        52.39643345, 71.44442849, 52.70405102, 72.93717623,  8.48674574,
        12.08341818,  9.39201493, 11.59162741,  8.70822477, 11.47542834,
        16.56751709, 21.64719272, 16.39411807, 23.08985319, 17.28915339,
        24.07483945, 56.56345105, 